## Dask Sentinel-2 cloudless

Install the `eopf-sentinel-2` using `pip`

In [ ]:
%%bash 
cd /workspace/dask-app-package/eopf-sentinel-2 
pip install -e .

Import the additional libraries:

In [1]:
import os
from click.testing import CliRunner
from dask_gateway import Gateway
from eopf_sentinel_2.app import main, start

## Dask Gateway connection

The environment variable `DASK_GATEWAY_ADDRESS` contains the Dask Gateway internal service address.

Create a `Gateway` object that will later be used to create a Dask cluster

In [2]:
gateway_url = os.environ.get("DASK_GATEWAY_ADDRESS")

gateway = Gateway(gateway_url)



## Create a Dask cluster

The environment variable `DASK_IMAGE` contains the container image to use as for the Dask cluster workers.

This might take a few minutes.

In [ ]:
image = os.environ.get("DASK_IMAGE")

In [3]:
cluster_options = gateway.cluster_options()


worker_cores = 0.5
worker_cores_limit = 5
worker_memory = 4

if image == "":
    raise ValueError("Please set the image name")

cluster_options['image'] = image
cluster_options['worker_cores'] = worker_cores
cluster_options['worker_cores_limit'] = worker_cores_limit
cluster_options['worker_memory'] = f"{worker_memory} G"

In [4]:
cluster = gateway.new_cluster(cluster_options)

To monitor the progress in a shell, use:

```
kubectl get pods -l 'app.kubernetes.io/component=dask-scheduler'
```

Print the cluster name:


In [ ]:
cluster.name

Print the cluster dashboard link although it's not accessible via browser.

In [ ]:
cluster.dashboard_link

If there's a port forward on the `traefik-dask-gateway` service, the dashboard link becomes:

In [ ]:
print(f"http://localhost:8001/clusters/{cluster.name}/status")

To use the cluster in a terminal, set the `DASK_CLUSTER` environment variable

In [ ]:
print(f"export DASK_CLUSTER='{cluster.name}'")

Get the cluster client and scale the cluster workers:

In [ ]:

client = cluster.get_client()

cluster.adapt(minimum=4, maximum=6)

To monitor the progress in a shell, do: 

```
kubectl get pods -l app.kubernetes.io/component=dask-worker
```

## EOPF Sentinel-2 processing

### Using the main function


In [ ]:
params = {
    "item_url": "https://stac.core.eopf.eodc.eu/collections/sentinel-2-l1c/items/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458"
}

main(**params)

### Using the command-line interface

In [ ]:
runner = CliRunner()
result = runner.invoke(start, ['--help'])

print(result.output)

Create the `DASK_CLUSTER` environment variable with the Dask cluster name

In [ ]:
os.environ["DASK_CLUSTER"] = cluster.name

arguments = ["--item-url", "https://stac.core.eopf.eodc.eu/collections/sentinel-2-l1c/items/S2B_MSIL1C_20250113T103309_N0511_R108_T32TLQ_20250113T122458"]

runner = CliRunner()
result = runner.invoke(start, args=arguments)

print(result.output)

## Dispose the cluster

In [ ]:
cluster.shutdown()